In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
# import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
# torch.manual_seed(SEED)
# torch.use_deterministic_algorithms(True)

#### The Data

In [3]:
# train_df = pd.read_csv('../../final/data/train_set_basic.csv')
train_df = pd.read_csv('../../../anemia_ml4hc/data/train_set_basic.csv')
#train_df = pd.read_csv('../../final/data/train_set_noisy_6_missing_3.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,14.728733,-1.000000,3.170892,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,44.186200,-1.0,0
1,10.405752,9.634615,5.659537,-1.000000,-1.000000,77.413788,212.671838,4.032519,0,0.887130,96.311597,-1.0,43.218595,-1.0,83.207518,31.217256,-1.0,4
2,15.132737,358.914888,1.842252,3.797487,315.102272,80.500314,-1.000000,5.639507,0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,45.398211,-1.0,0
3,11.340169,-1.000000,1.662209,2.441767,-1.000000,97.033963,102.079062,3.506041,1,1.020527,127.281715,-1.0,20.847013,-1.0,62.210273,34.020508,-1.0,6
4,6.691485,-1.000000,3.337971,-1.000000,-1.000000,99.838438,24.119564,2.010694,0,1.957666,34.633063,-1.0,34.612121,-1.0,112.411298,20.074456,-1.0,5


In [4]:
train_df.label.value_counts()

0    7200
6    6501
5    6498
1    6483
2    6454
3    6378
4    6047
7    4839
Name: label, dtype: int64

In [5]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [6]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((50400, 17), (14000, 17), (50400,), (14000,))

#### Training 

In [7]:
# %%time
# timesteps = int(1e6)
# dqn_model = utils.stable_dqn(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [8]:
from stable_baselines import DQN
from stable_baselines import bench, logger
from stable_baselines.common.callbacks import CheckpointCallback
import tensorflow
tensorflow.set_random_seed(constants.SEED)

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWa

In [10]:
# for steps in [int(50e6)]:
#     dqn_model = utils.stable_dqn(X_train, y_train, steps, True, 
#                                  f'../../../anemia_ml4hc/models/complete_models/dueling_ddqn_{steps}',
#                                  checkpoint_folder = '../../../anemia_ml4hc/models/sb/dueling_double_dqn',
#                                  checkpoint_prefix='dddqn_basic')
    
for steps in [int(10e6), int(15e6)]:
# for steps in [int(2e3)]:
    dqn_model = utils.stable_vanilla_dqn(X_train, y_train, steps, True, 
                                         f'../../../anemia_ml4hc/models/sb/prioritized_replay/dqn_{steps}')#,
#                                        checkpoint_folder = '../../../anemia_ml4hc/models/sb/dqn/logs/',
#                                        checkpoint_prefix='dqn_basic')

using just stable baselines (not 3)
--------------------------------------
| % time spent exploring  | 87       |
| episodes                | 50000    |
| mean 100 episode reward | -0.8     |
| steps                   | 136810   |
| success rate            | 0.13     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 72       |
| episodes                | 100000   |
| mean 100 episode reward | -0.7     |
| steps                   | 286432   |
| success rate            | 0.18     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 58       |
| episodes                | 150000   |
| mean 100 episode reward | -1       |
| steps                   | 439881   |
| success rate            | 0.06     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 43       |
| episodes                | 200000   |
| mean 100 episode reward | 

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1550000  |
| mean 100 episode reward | -0.8     |
| steps                   | 4029570  |
| success rate            | 0.1      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1600000  |
| mean 100 episode reward | -0.9     |
| steps                   | 4085660  |
| success rate            | 0.06     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1650000  |
| mean 100 episode reward | -0.7     |
| steps                   | 4141998  |
| success rate            | 0.17     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1700000  |
| mean 100 episode reward | -0.8     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3100000  |
| mean 100 episode reward | -0.7     |
| steps                   | 5805234  |
| success rate            | 0.13     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3150000  |
| mean 100 episode reward | -0.8     |
| steps                   | 5863162  |
| success rate            | 0.09     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3200000  |
| mean 100 episode reward | -0.8     |
| steps                   | 5921601  |
| success rate            | 0.11     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3250000  |
| mean 100 episode reward | -0.6     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4650000  |
| mean 100 episode reward | -0.9     |
| steps                   | 7597293  |
| success rate            | 0.07     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4700000  |
| mean 100 episode reward | -0.5     |
| steps                   | 7652703  |
| success rate            | 0.24     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4750000  |
| mean 100 episode reward | -0.7     |
| steps                   | 7712661  |
| success rate            | 0.15     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4800000  |
| mean 100 episode reward | -0.8     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6200000  |
| mean 100 episode reward | -0.8     |
| steps                   | 9381357  |
| success rate            | 0.12     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6250000  |
| mean 100 episode reward | -0.8     |
| steps                   | 9437497  |
| success rate            | 0.14     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6300000  |
| mean 100 episode reward | -0.7     |
| steps                   | 9494681  |
| success rate            | 0.13     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6350000  |
| mean 100 episode reward | -0.8     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1000000  |
| mean 100 episode reward | -1       |
| steps                   | 3112323  |
| success rate            | 0.12     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1050000  |
| mean 100 episode reward | -1       |
| steps                   | 3270715  |
| success rate            | 0.11     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1100000  |
| mean 100 episode reward | -0.9     |
| steps                   | 3439823  |
| success rate            | 0.12     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 1150000  |
| mean 100 episode reward | -1       |
| steps                  

KeyboardInterrupt: 

#### Testing

#### Saving files

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/test_df_2800000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/success_df_2800000.csv', index=False)